# Mixed Discrete - Continuous optimization

Mixed discrete-continuous optimization occurs in a variety of scientific and engineering fields. In these optimization one usually wants to solve a set of non linear equations using continuous varialbles that are subjected to some parameters usually defined as a set of possible discrete values. The problem usually consists of solving the continuous equation while minimizing a cost function related to the discrete variables while ensuring some propoerties of the continuout variables.

For example one wants to solve hydraudynamic equation of a wing, while minimizing its cost but ensuring that the overall drag is above a certain thereshold. [find better example]

To illustrate this problem class and how to solve them using a QUBO let's consider the following set of non-linear of equations:

$$
-1 - x_0 + x_1 = 0 \\
 1 - x_1 = 0 \\
 2 - q x_0^2 - x_2 = 0 \\
 -p x_1^2 + x_2 - x_3 = 0 
$$

Where $x_i$ are continuous variables and $p$ and $q$ are discrete parameters.

The goal is for example to find valied solutions of this system reprented by values of the $x_i$ variables while minimizing the sum $p+q$ but making sure that the values of $x_3$ is within a certain range. 

Let's first start by exploring classicaly this toy problem. We first need to define a function that returns the solution of the problem for a ginve input and parameter values. (We assume here tha `parameters` is a global variable)

In [1]:
import numpy as np

def nlfunc(input):
    x0,x1,x2,x3 = input
    q, p = parameters

    def f0():
        return -1 - x0 + x1
    
    def f1():
        return 1 - x1
    
    def f2():
        return 2  - q*x0**2  - x2

    def f3():
        return -p*x1**2 + x2 - x3
    
    return np.array([f0(), f1(), f2(), f3()])



## Classical Solution

We can solve the problem for every combination of $p$ and $q$ values. We assume that $p$ and $q$ can take the values: 0,1, 2 or 3.

In [2]:
from quantum_newton_raphson.newton_raphson import newton_raphson

parameters_list = [[i, j] for i in range(4) for j in range(4)] 

for parameters in parameters_list:
    initial_point = np.random.rand(4)
    res = newton_raphson(nlfunc, initial_point)
    assert np.allclose(nlfunc(res.solution), 0)
    print(parameters, res.solution)

[0, 0] [-4.85753346e-17  1.00000000e+00  2.00000000e+00  2.00000000e+00]
[0, 1] [-4.85713275e-17  1.00000000e+00  2.00000000e+00  1.00000000e+00]
[0, 2] [-1.73437802e-21  1.00000000e+00  2.00000000e+00  3.39173134e-14]
[0, 3] [-1.15647772e-16  1.00000000e+00  2.00000000e+00 -1.00000000e+00]
[1, 0] [1.95363259e-21 1.00000000e+00 2.00000000e+00 2.00000000e+00]
[1, 1] [1.11021259e-16 1.00000000e+00 2.00000000e+00 1.00000000e+00]
[1, 2] [ 8.32695877e-17  1.00000000e+00  2.00000000e+00 -2.57160959e-11]
[1, 3] [ 9.25152063e-18  1.00000000e+00  2.00000000e+00 -1.00000000e+00]
[2, 0] [-1.11021998e-16  1.00000000e+00  2.00000000e+00  2.00000000e+00]
[2, 1] [1.11021846e-16 1.00000000e+00 2.00000000e+00 1.00000000e+00]
[2, 2] [-5.55102610e-17  1.00000000e+00  2.00000000e+00 -2.14738227e-11]
[2, 3] [-3.70059129e-17  1.00000000e+00  2.00000000e+00 -1.00000000e+00]
[3, 0] [2.07888583e-21 1.00000000e+00 2.00000000e+00 2.00000000e+00]
[3, 1] [1.11023674e-16 1.00000000e+00 2.00000000e+00 1.00000000e+00

/home/nico/QuantumApplicationLab/QuantumNewtonRaphson/quantum_newton_raphson/utils.py:74: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn("spsolve requires A be CSC or CSR matrix format", SparseEfficiencyWarning)


# QUBO Solution with constraints on the parameters.

We have explained in the previous notebook how to solve polynomial system of equations using QUBO. Here we not only want to solve the equation but also find the solution that corresponds to for example the maximal value of $p+q$. We therefore extand the system of equations with an additional equation: 

$$
-1 - x_0 + x_1 = 0 \\
 1 - x_1 = 0 \\
 2 - q x_0^2 - x_2 = 0 \\
 -p x_1^2 + x_2 - x_3 = 0 \\
 c(M - q - p) \rightarrow 0 
$$

The last equation corresponds to the minimization of the sum  $p+q$. The constant $c$ is a weight toadjust the relative importance of the last equation wrt the other four. $M$ is the maximal value that can take $p+q$. We now that upfront since the variables $p$ and $q$ can only take the values: 0, 1, 2,3, and therefore $M=6$.

As for the simple polynomial case we need to create the matrices of this polynomial system. This is done as below:

In [24]:
import numpy as np
import sparse
def define_matrices():
    
    # system of equations
    num_equations = 5
    num_variables = 6

    c = 0.5*1E-1
    M = 6

    P0 = np.zeros((num_equations,1))
    P0[0] = -1
    P0[1] = 1
    P0[2] = 2
    P0[3] = 0
    P0[4] = c*M

    P1 = np.zeros((num_equations, num_variables))
    P1[0, 0] = -1
    P1[0, 1] =  1

    P1[1, 1] = -1

    P1[2, 2] = -1

    P1[3, 2] =  1 
    P1[3, 3] = -1

    # cost
    P1[4,4] = -c
    P1[4,5] = -c
   

    P2 = np.zeros((num_equations, num_variables, num_variables))


    P3 = np.zeros((num_equations, num_variables, num_variables, num_variables))
    P3[2, 0, 0, 4] = -1
    P3[3, 1, 1, 5] = -1


    return sparse.COO(P0), sparse.COO(P1), sparse.COO(P2), sparse.COO(P3)

matrices = define_matrices()

To solve he problem we first need to encode the different variables in a series if qubits.

In [25]:
from qubops.encodings import PositiveQbitEncoding
from qubops.solution_vector import SolutionVector_V2 as SolutionVector
from qubops.mixed_solution_vector import MixedSolutionVector_V2 as MixedSolutionVector


# define the encoding for the first four varialbes (x0, x1, x2, x3)
nqbit = 2
step = 1
encoding1 = PositiveQbitEncoding(nqbit = nqbit, step=step, offset=-1, var_base_name='x')
sol_vec1 = SolutionVector(4, encoding=encoding1)

# define the encoding for the last two variables (p, q)
nqbit = 2
step = 1
encoding2 = PositiveQbitEncoding(nqbit = nqbit, step=step, offset=0, var_base_name='x')
sol_vec2 = SolutionVector(2, encoding=encoding2)

# define the solution vector
sol_vec = MixedSolutionVector([sol_vec1,sol_vec2])

The endoding for the continuous varialbes can take the following values: 

In [26]:
np.sort(encoding1.get_possible_values())

array([-1,  0,  1,  2])

While the encoding for the discrete variales can take the values:

In [27]:
np.sort(encoding2.get_possible_values())

array([0, 1, 2, 3])

We can now solve the problem

In [28]:
from dwave.samplers import SteepestDescentSolver
from qubops.qubops_mixed_vars import QUBOPS_MIXED
options = {'num_reads':1000, 'sampler':SteepestDescentSolver()}
qubops = QUBOPS_MIXED(sol_vec, options)

sol = qubops.solve(matrices)
sol

[[0, 1, 2, -1], [3, 3]]

The solver finds the correct solution $[0,1,2,-1]$ that corresponds to the minimum value of the sum $p+q=6$

# QUBO Solution with constraints on the parameters and on the solution.

We can now introduce a restriction on the acceptable values of some of the solution elements. We for example could specify that the value of $x_3$ should be between 0 and 1. We then wants to solve the non-linear system, while maximizing $p+q$ and ensuring that $1\leq x_3 \leq 2$. 

As we can see from the classical exploration of all the solution above, the explected solution is therefore $[0,1,2,0]$ obtaind for $p=3$ and $q=2$.

Such constraints can  included in the QUBO model as an additional penalty term of the QUBO cost function.    

In [56]:
qubops = QUBOPS_MIXED(sol_vec, options)
bqm = qubops.create_bqm(matrices, strength=1E4)
slacks1 = bqm.add_linear_inequality_constraint(qubops.all_expr[3], lagrange_multiplier=1, label="const", lb=1, ub=2)

In [57]:
sampleset = qubops.sample_bqm(bqm, num_reads=1000)

In [58]:
slowest_sol = sampleset.lowest()
sol = qubops.decode_solution(slowest_sol.record[0][0])
sol

[[0, 1, 2, 0], [3, 2]]